# Descrizione del problema
Il problema riguarda lo **scheduling**. Sono dati $G$ ingegneri che lavorono per $L$ giorni. La realtà è composta da 3 elementi:
* **Features**: sono le funzionalità implementate nei prodotti;
* **Services**: sono i programmi che sono in esecuzione sui server;
* **Binaries**: modellano un server. Ogni *binary* può avere in esecuzione più servizi;

## Relazioni tra le entità
Una **Feature** è contraddistinta dal numero di utenti che riesce a soddisfare e da un insieme di servizi in cui deve essere implementata. Una **Feature** è disponibile ai clienti quando viene implementata in ogni servizio da cui dipende.
Un **Service** è presente su **uno e un solo binary**, ma su ogni **binary** è possibile inserire più **Services**.

In [1008]:
# modellazione del sistema
from dataclasses import dataclass
from typing import List, Tuple, Dict

score: int = 0
daily_schedule: List[Tuple[int, int]] = [] # giorno, binary

@dataclass
class Binary:
    id: int
    services: List[str]
    working_on: int
    locked: bool

@dataclass
class Service:
    name: str
    binary: int
    impl_features: List[str]
    
@dataclass
class Feature:
    idx: int
    name: str
    difficulty: int
    users: int
    dep_services: List[List]
    completed: bool

@dataclass
class Engineer:
    id: int
    avail: int
    ops: List[Tuple[str, List]]

    def add_op(self, op_name: str, day: str, duration: int, values: List[int]):
        if day > self.avail:
            self.ops.append(("wait", [day - self.avail]))

        affected_binaries = []
        if op_name == "move":
            service = services[values[0]]
            b1 = service_to_binaries[service.name]
            affected_binaries.append(b1, values[1])
        elif op_name == "impl":
            affected_binaries.append(values[1])
        else:
            raise Exception(f"operation {op_name} not supported")

        # controllo se un binary coinvolto nell'operazione è bloccato
        # se si aggiungo una wait pari al massimo di quelli schedulati
        locked = [b for b in affected_binaries if binaries[b].locked]
        skip = []
        for l in locked:
            for ds in daily_schedule:
                if daily_schedule[ds] == l:
                    skip.append(ds)
        else:
            if len(skip) > 0:
                self.ops.append("wait", [day + skip])

        self.avail += day + duration
        self.ops.append((op_name, values))

## Lettura input
In questa sezione, viene effettuato il parsing dei dati in ingresso creando:
* Lista dei binary: tutti i binary del sistema; per ogni binary si tiene traccia della list dei service installati e del numero di ingegneri che lavora al momento;
* Lista dei service; tutti i service del sistema. Per ogni service si tiene conto delle feature implementate al momento;
* Lista delle feature: tutte le feature del sistema. Ogni feature contiene una lista coppie (stringhe, booleani) in cui si tiene conto dei servizi da implementare e del loro stato (implementato si/no);

In [1009]:
fname = "data/f.txt"

# lettura dei dati dal file di input
with open(fname, "r") as f:
    lines = f.readlines()
    
# parsing della prima riga: metadati del problema
L, G, S, B, F, N = [int(k) for k in lines[0].strip().split(" ")]

binaries: List[Binary] =  [None] * B
service_to_binaries: Dict[str, int] = {}
services: List[Service] = []
features: List[Feature] = []
engineers: List[Engineer] = []

for i in range(G):
    engineers.append(Engineer(i, 0, []))

for (service, binary) in [line.strip().split() for line in lines[1:S+1]]:
    bid = int(binary)
    b = binaries[bid]
    if b is None:
        b = Binary(bid, [service], 0, False)
        binaries[bid] = b
    else:
        b.services.append(service)

    s = Service(service, bid, [])
    services.append(s)
    service_to_binaries[service] = bid
idx = 0
for i, feature in enumerate(lines[1+S:]):
    # leggo solo le righe pari
    if i % 2 ==0:
        feature, num_services, difficulty, users = feature.strip().split(" ")
        s = [[s, False] for s in lines[1+S+1+i].split()]
        f = Feature(idx, feature, int(difficulty), int(users), s, False)
        features.append(f)
        idx += 1

print(engineers, binaries, service_to_binaries, features, services, sep="\n")

[Engineer(id=0, avail=0, ops=[]), Engineer(id=1, avail=0, ops=[]), Engineer(id=2, avail=0, ops=[]), Engineer(id=3, avail=0, ops=[]), Engineer(id=4, avail=0, ops=[]), Engineer(id=5, avail=0, ops=[]), Engineer(id=6, avail=0, ops=[]), Engineer(id=7, avail=0, ops=[]), Engineer(id=8, avail=0, ops=[]), Engineer(id=9, avail=0, ops=[])]
[Binary(id=0, services=['sdd', 'sbbe', 'scie', 'scif', 'sdag', 'sdde', 'seef'], working_on=0, locked=False), Binary(id=1, services=['sde', 'sid', 'sdee'], working_on=0, locked=False), Binary(id=2, services=['sbaf', 'sbdg', 'sbgf', 'scbc', 'scfd', 'sdae', 'segf'], working_on=0, locked=False), Binary(id=3, services=['sge', 'scfc', 'sdeg', 'sdfg', 'sdje'], working_on=0, locked=False), Binary(id=4, services=['sfaa'], working_on=0, locked=False), Binary(id=5, services=['sdie', 'seba', 'sebi', 'secg'], working_on=0, locked=False), Binary(id=6, services=['seg', 'sbie', 'sbif', 'scdc', 'scgb', 'sche', 'sdce', 'sdjf', 'sejb'], working_on=0, locked=False), Binary(id=7, s

## Attività
Di seguito sono illustrate le mosse che un ingegnere può effettuare.

Ciascuna mossa è caratterizzata da una durata. Per ogni mossa è definita una procedura per eseguirla sui dati di input. Ogni procedura ha la seguente firma:
* input:
    * ingegnere su cui schedulare l'attività;
    * giorno in cui effettuare l'attività;
    * feature, binary, service su cui effettuare l'attività;
* output:
    * durata dell'attività;

### Implementazione di una nuova feature
Su un *binary* $B_j$ può un ingegnere implementare tutti i servizi per una feature $F_i$. Questa mossa ha una durata $D_{Fi} + R_{Bj} + C_{Bj}$. Dove $D_{Fi}$ è data dalla complessità della **Feature**, $R_{Bj}$ è dato dal numero di servizi totali presenti nel *binary*, $C_{Bj}$ ingegneri che stanno già lavorando su $B_j$ nel giorno in cui viene *schedulata* l'attività. Inoltre, ogni ingegnere può implementare la stessa **Feature** su **binary** diversi. Infine, se più ingegneri sono *schedulati* per lavorare sullo stesso **binary** il primo della soluzione viene schedulato e conta per gli altri nel fattore $C_{Bj}$;


In [1010]:
def implement_feature(e: Engineer, day: int, f: int, b: int) -> int:
    assert(b >= 0 and b < B)
    assert(day < L  and day >=0)
    global score
    feature = features[f]
    duration  = feature.difficulty + len(binaries[b].services) + binaries[b].working_on
    for service in feature.dep_services:
        s_name = service[0]
        done = service[1]
        if service_to_binaries[s_name] == b and not done:
            service[1] = True
    
    for (service, done) in feature.dep_services:
        if not done:
            break
    else:
        feature.completed = True
        # print("DEBUG: completed", feature, "multiply", 1 if  L-(day + duration) >= 0 else 0  )
        score += feature.users * 1 if  L-(day + duration) >= 0 else 0 

    e.add_op("impl", day, duration, [f, b])
    daily_schedule.append((day + duration, b))
    binaries[b].working_on+=1
    
    return duration

### Migrazione di un service
un ingegnere può spostare un servizio $S_i$ da un *binary* $B_j$ a $B_k$, mantenendo le funzionalità implementate. Per fare questa mossa, un ingegnere impiega $\max (R_{bj}, R_{bk})$ giorni dove $R_{bj}, R_{bk}$ sono i servizi in esecuzione rispettivamente su $B_j$ a $B_k$. Quest'azione però è bloccante: nessun ingegnere può lavorare su $B_j$ e $B_k$ quando è in corso una migrazione; 

In [1011]:
def move_service(e: Engineer, day: int, s: int, dst: int) -> int:
    assert(service_to_binaries[s] != dst)
    assert(day < L and day >=0)
    assert(dst >= 0 and dst < B)
    service = services[s]
    src = service.binary
    duration  = max(len(binaries[src]), len(binaries[dst]))
    service_to_binaries[service.name] = dst
    binaries[src].remove(s)
    binaries[dst].append(s)
    binaries[src].locked = True
    binaries[dst].locked = True
    e.add_op("move", day, duration, [s, dst])
    return duration

### Nuovo binary
Un ingegnere può creare un nuovo binary senza nessun servizio al suo interno. Il costo per creare è un nuovo binary è fisso e vale $N$.


In [1012]:
def new_binary(e: Engineer, day: int)-> int:
    binaries.append(Binary(id))
    e.add_op("new", day, N, [])
    return N

### Idle
Un ingegnere può essere in idle per un numero di giorni $1\leq W \leq L$

In [1013]:
def idle(e: Engineer, day: int, duration: int) -> int:
    e.add_op("wait", day, duration, [])
    return 1

# Approccio 1
In una fase preliminare, l'algoritmo ordina le feature per numero di utenti soddisfatti. In questo approccio, si assegna un punteggio maggiore ai servizi che risiedono sulla stessa macchina.

In [1014]:
import random

def get_todo(features: List [Feature]) -> List[Feature]:
    res = []
    for feature in features:
        if not feature.completed:
            res.append(feature)
    return res

for day in range(L):
    while len(daily_schedule) > 0 and daily_schedule[-1][0] <= day:
        bid: int = daily_schedule[-1][1] 
        binaries[bid].working_on -=1
        binaries[bid].locked = False
        daily_schedule.pop()
    
    todo = get_todo(features)
    if len(todo) == 0:
        break

    for engineer in engineers:
        if engineer.avail <= day:
            todo = get_todo(features)
            if len(todo) == 0:
                break
            b = random.choice(binaries)
            f = random.choice(todo)
            # print("impl", f.name, b)
            implement_feature(engineer, day, f.idx, b.id)


for engineer in engineers:
    print(engineer.id)
    for op in engineer.ops:
        print(op[0], end=" ")
        f = features[op[1][0]]
        print(f.name, op[1][1])

print(score)
        

0
impl sjjjjidc 40
impl sjcajcja 54
impl sddegaih 97
impl scggfggd 22
1
impl sfgdghjh 21
impl sfjdbcic 74
impl sjjfcieh 89
impl sfdgjfci 21
2
impl sfheadhb 59
impl sdiedadh 81
impl sfdedabc 92
3
impl sgghjjii 16
impl siaggdjh 56
impl sidhcbge 22
4
impl shjehjbj 52
impl scidjegj 74
impl sgeejgjf 22
impl sjjjbbac 72
5
impl shiihab 92
impl sgbjchah 87
impl sjhfibd 91
impl sdiedeb 80
6
impl sccbbdid 13
impl scaebjff 49
impl sgbcebjh 91
impl sfchigch 68
7
impl siicdbja 18
impl sifahdgd 45
impl sbddedeb 92
impl scaaafh 10
8
impl sdgbhaa 56
impl sjhhfbf 51
impl sjjbefbh 37
9
impl sggfcbfh 89
impl shdafgja 43
impl sdfhcjgd 47
impl sgaigbfg 5
0
